# K-Nearest Neighbors {#sec-knn}

## Overview

In this section we will dive into one of the simplest algorithms to perform classification. Namely, the
<a href="https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm">k-nearest neighbors algorithm</a> or kNN for short.
This is a non-parametric supervised learning method that it can be used for clustering and regression apart from classification. 

We will limit ourselves to the case of classification herein. In the case of classification, the input to
the algorithm is a dataset $\mathbf{D}$ of data points alongside their labels $Y$ and a positive integer $k$.
The algorithm tries to find the $k$ most similar data points in $\mathbf{D}$ in order to produce the classification output.

The kNN algorithm, apart from classification, is also used as an intermediate steps in algorithms such as <a href="https://en.wikipedia.org/wiki/Spectral_clustering">Laplacian spectral embedding</a>, <a href="https://en.wikipedia.org/wiki/Nonlinear_dimensionality_reduction">locally linear embedding</a> and <a href="https://en.wikipedia.org/wiki/Isomap">Isomap</a>.

## K-Nearest Neighbors

The kNN algorithm is fairly is to grasp. The input is a dataset $\mathbf{D}$ ,a positive integer $k$ and a
distance function that allows us to compute similarities between the points in the dataset.
The algorithm tries to find the $k$ most similar data points in $\mathbf{D}$ in order to produce the classification output.
A data point is classified using a plurality vote of its neighbors; the object is assigned the class most common among its $k$ nearest neighbors. If $k = 1$, then the object is simply assigned to the class of that single nearest neighbor.

Thus, the vanilla KNN algorithm does not train any parameters although we will have to experiment 
on a different number of neighbors and possibly the distance function to use. The algorithm stores the entire
dataset $\mathbf{D}$ in memory something that can be really problematic for large datasets. In addition, the vanilla
algorithm for a dataset with $N$ points each having $M$ features, requires $O(MN^2)$ time which again for large datasets can be prohibitive. 
Implementations therefore of the KNN algorithm use <a href="https://en.wikipedia.org/wiki/K-d_tree">k-d$ trees</a>, see e.g. [2] or <a href="https://en.wikipedia.org/wiki/Ball_tree">ball trees</a> however, we will not go into details. 


The kNN algorithm requires the number of neighbors to use as an input. So one may ask what is the best choice for it?
This really depends on the data. In general, larger values of $k$ reduce the  effect of noise but make boundaries between classes less distinct. 
Hyperparameter optimzation can be used in order to determine $k$. When dealing with only two classes i.e. binary classification, it is better to choose an odd $k$ this avoids tied votes. Boostrap can also be employed in this setting in order to find the optimal $k$ [1].

A commonly used distance metric for continuous variables is Euclidean distance. However, other metrics can be used. 
Additionally, the performance of the algorithm can be significantly improved if the distance metric is learned with specialized algorithms such as 
<a href="https://en.wikipedia.org/wiki/Large_margin_nearest_neighbor">large margin nearest neighbor</a> or <a href="https://en.wikipedia.org/wiki/Neighbourhood_components_analysis">neighbourhood components analysis</a> [1].
In addition, in order to improve the accuracy of the algorithm is to assign weights to the contributions of the neighbors, so that the nearer neighbors contribute more to the average than the more distant ones. For example, a common weighting scheme consists in giving each neighbor a weight of $1/d$, where $d$ is the distance to the neighbor [1]. This can be beneficial when there class distribution is skewed.

### Disadvantages

kNN is a very simple algorithm and this is one of the reasons why it is employed frequently in practice. However, the
algorithm has a number of disadvantages that one should be aware of in order to avoid misuses of the method. These are listed below.

Since this algorithm relies on distance for classification, if the features represent different
physical units or come in vastly different scales then normalizing the training data can improve its accuracy dramatically [1].
This is true for every algorithm that employes distances. Furthermore, the presence of noisy or irrelevant features can also severely diminish the accuracy of the algorithm. Moreover, the vanilla _majority voting_ classification can be problematic occurs when the class distribution is skewed. 
In this case, the  examples of the more frequently occuring class in  $\mathbf{D}$, tend to dominate the prediction of the new data point.
One approach that can be used in order to address this problem, is weighting the contribution of  each neighbor according to the distance computed.
Note however, that many algorithms struggle when class imbalances are very dominat.

#### Example 1

In this example we will use 
the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html">KNeighborsClassifier</a> 
in scikit-learn in order to perform classification on the Iris dataset.

In [13]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import matplotlib.pyplot as plt

In [7]:
X, y = load_iris(return_X_y=True)
num_classes = len(set(y))
print(f"We have {y.size} labeled examples across the following "
      f"{num_classes} classes:\n{set(y)}\n")
print(f"First four feature rows:\n{X[:4]}")
print(f"\nFirst four labels:\n{y[:4]}")

We have 150 labeled examples across the following 3 classes:
{0, 1, 2}

First four feature rows:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]]

First four labels:
[0 0 0 0]


In [8]:
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)
print(f"Training set labels:\n{y_train}")

Training set labels:
[0 0 2 1 1 0 0 1 2 2 1 2 1 2 1 0 2 1 0 0 0 1 2 0 0 0 1 0 1 2 0 1 2 0 2 2 1
 1 2 1 0 1 2 0 0 1 1 0 2 0 0 1 1 2 1 2 2 1 0 0 2 2 0 0 0 1 2 0 2 2 0 1 1 2
 1 2 0 2 1 2 1 1 1 0 1 1 0 1 2 2 0 1 2 2 0 2 0 1 2 2 1 2 1 1 2 2 0 1 2 0 1
 2]


In [9]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [11]:
predictions = knn.predict(X_test)
target_names=['0', '1', '2']
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00        12

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



Let's also use the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html">GridSearchCV</a> to search for the optimal number of $k$.

In [17]:
n_neighbors = np.array([i for i in range(1, 10)])
param_grid = dict(n_neighbors=n_neighbors)
grid = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid)
grid.fit(X_train, y_train)
print(f"GridSearch best score {grid.best_score_}")
print(f"GridSearch number of neighbors for best estimator  {grid.best_estimator_.n_neighbors}")

GridSearch best score 0.9644268774703558
GridSearch number of neighbors for best estimator  4


## Summary

In this section, we reviewd the kNN algorithm. This is a very simple distance-based algorithm. The algorithm was discussed in the context of
classification however is can be used for regression also. The algorithm tries to find the $k$ most similar data points in $\mathbf{D}$ in order to produce the classification output. A data point is classified using a plurality vote of its neighbors; the object is assigned the class most common among its $k$ nearest neighbors. If $k = 1$, then the object is simply assigned to the class of that single nearest neighbor.
The vanilla kNN algorithm does not train any parameters although we will have to experiment 
on a different number of neighbors and possibly the distance function to use. The algorithm stores the entire
dataset $\mathbf{D}$ in memory something that can be really problematic for large datasets. In addition, the vanilla
algorithm for a dataset with $N$ points each having $M$ features, requires $O(MN^2)$ time which again for large datasets can be prohibitive. 
Implementations therefore of the KNN algorithm use <a href="https://en.wikipedia.org/wiki/K-d_tree">k-d$ trees</a>, see e.g. [2] or <a href="https://en.wikipedia.org/wiki/Ball_tree">ball trees</a>.

Since this algorithm relies on distance for classification, if the features represent different
physical units or come in vastly different scales then normalizing the training data can improve its accuracy dramatically [1].
This is true for every algorithm that employes distances. Furthermore, the presence of noisy or irrelevant features can also severely diminish the accuracy of the algorithm. Moreover, the vanilla _majority voting_ classification can be problematic occurs when the class distribution is skewed. 
In this case, the  examples of the more frequently occuring class in  $\mathbf{D}$, tend to dominate the prediction of the new data point.
One approach that can be used in order to address this problem, is weighting the contribution of  each neighbor according to the distance computed.
Note however, that many algorithms struggle when class imbalances are very dominat.

We also saw how to use  the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html">KNeighborsClassifier</a> class in order to classify data points from the Iris dataset. In addition, we performed a grid search using the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html">GridSearchCV</a> class.

## References

1. <a href="https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm">k-nearest neighbors algorithm</a>
2. Marcello La Rocca, _Advanced algorithms and data structures_, Manning Publications.